In [ ]:
import os

### Tách chuỗi AMP

In [ ]:
POSITIVE = {'K', 'R'}
NEGATIVE = {'D', 'E'}
NON_POLAR = {'G', 'A', 'V', 'L', 'I', 'M', 'P', 'F', 'W'}

def calculate_net_charge(seq):
    return sum(1 for aa in seq if aa in POSITIVE) - sum(1 for aa in seq if aa in NEGATIVE)

def calculate_hydrophobic_content(seq):
    non_polar_count = sum(1 for aa in seq if aa in NON_POLAR)
    return round((non_polar_count / len(seq)) * 100, 2)

def analyze_peptide_top5(peptide):
    results = []
    for length in range(10, 51):  # Ưu tiên đoạn dài hơn
        for start in range(len(peptide) - length + 1):
            sub = peptide[start:start+length]
            charge = calculate_net_charge(sub)
            hydro = calculate_hydrophobic_content(sub)

            if charge >= 3 and hydro >= 60:
                results.append({
                    'start': start + 1,
                    'end': start + length,
                    'length': length,
                    'sequence': sub,
                    'net_charge': charge,
                    'hydrophobic_%': hydro,
                })

    # Sắp xếp các peptide theo độ hydrophobic, net_charge và chiều dài
    sorted_results = sorted(results, key=lambda x: (x['hydrophobic_%'], x['net_charge'], x['length']), reverse=True)

    top = []
    for res in sorted_results:
        if not any(res['start'] <= existing['end'] and res['end'] >= existing['start'] for existing in top):
            top.append(res)
        elif len(top) < 5:
            for existing in top:
                if existing['start'] <= res['end'] and existing['end'] >= res['start']:
                    if res['length'] < existing['length']:
                        top.remove(existing)
                        top.append(res)
                        break
        if len(top) == 5:
            break
    return top

def process_fasta_file(input_path, output_path, resume_from_line=0, batch_size=1000):
    existing_sequences = set()  # Khởi tạo `existing_sequences` trước khi sử dụng
    current_id = 1  # Biến lưu trữ ID hiện tại

    # Đọc các chuỗi đã có trong file output (nếu có) và lưu vào set `existing_sequences`, đồng thời lấy ID cuối cùng
    if os.path.exists(output_path):
        with open(output_path, 'r') as f:
            lines = f.readlines()
            for i in range(1, len(lines), 2):  # Đọc các chuỗi peptide đã có (dòng sau dấu '>')
                existing_sequences.add(lines[i].strip())
            # Lấy ID cuối cùng đã ghi vào file
            last_id_line = lines[-2] if len(lines) > 1 else None
            if last_id_line:
                current_id = int(last_id_line.split()[0].strip('>S_')) + 1  # Cập nhật ID tiếp theo

    sequence = ''
    line_count = resume_from_line  # Dòng hiện tại đã xử lý
    batch_to_write = []  # Danh sách lưu chuỗi peptide cần ghi vào file theo batch

    with open(input_path, 'r') as f:
        for current_line_num, line in enumerate(f):
            if current_line_num < resume_from_line:
                continue

            line = line.strip()
            if line.startswith('>'):
                # Khi gặp tiêu đề (dòng bắt đầu bằng '>')
                if sequence:
                    # Kiểm tra chuỗi peptide có độ dài <= 50
                    if len(sequence) <= 50:
                        if sequence not in existing_sequences:
                            batch_to_write.append(f">S_{current_line_num + 1} AMP\n{sequence}\n")
                            existing_sequences.add(sequence)  # Thêm vào existing_sequences để kiểm tra các lần sau
                    else:
                        # Nếu chuỗi dài hơn 50, tách thành các peptide con và kiểm tra
                        top_segments = analyze_peptide_top5(sequence)
                        for seg in top_segments:
                            seq = seg['sequence']
                            if seq not in existing_sequences:
                                batch_to_write.append(f">S_{current_line_num + 1} AMP\n{seq}\n")
                                existing_sequences.add(seq)  # Thêm vào existing_sequences để kiểm tra các lần sau
                sequence = ''
            else:
                sequence += line

            # Cập nhật số dòng đã xử lý
            line_count = current_line_num + 1  # Cập nhật dòng hiện tại đã xử lý

            # Khi batch đã đủ 5000 peptide, ghi vào file
            if len(batch_to_write) >= batch_size:
                with open(output_path, 'a') as f_output:
                    f_output.writelines(batch_to_write)
                batch_to_write.clear()  # Làm trống batch sau khi ghi
                with open('line_count.txt', 'w') as f_count:
                    f_count.write(str(line_count))
                print(f"Đã xử lý dòng: {line_count}")  # In ra số dòng hiện tại

    # Ghi các peptide còn lại vào file sau khi xử lý hết (nếu có batch còn lại)
    if batch_to_write:
        with open(output_path, 'a') as f_output:
            f_output.writelines(batch_to_write)

# Ví dụ sử dụng hàm
current_line = 0  # Bắt đầu từ dòng đầu tiên

# Tiến hành xử lý tệp và tiếp tục từ dòng đã xử lý trước đó
process_fasta_file('amp_not_genbank.fasta', 'separated_amp_not_genbank.fasta', resume_from_line=current_line)

### Tách chuỗi nAMP

In [ ]:
POSITIVE = {'K', 'R'}
NEGATIVE = {'D', 'E'}
NON_POLAR = {'G', 'A', 'V', 'L', 'I', 'M', 'P', 'F', 'W'}

def calculate_net_charge(seq):
    return sum(1 for aa in seq if aa in POSITIVE) - sum(1 for aa in seq if aa in NEGATIVE)

def calculate_hydrophobic_content(seq):
    non_polar_count = sum(1 for aa in seq if aa in NON_POLAR)
    return round((non_polar_count / len(seq)) * 100, 2)

def analyze_peptide_top5(peptide):
    results = []
    for length in range(10, 51):  # Ưu tiên đoạn dài hơn
        for start in range(len(peptide) - length + 1):
            sub = peptide[start:start+length]
            charge = calculate_net_charge(sub)
            hydro = calculate_hydrophobic_content(sub)

            if charge <3 and hydro <= 30:
                results.append({
                    'start': start + 1,
                    'end': start + length,
                    'length': length,
                    'sequence': sub,
                    'net_charge': charge,
                    'hydrophobic_%': hydro,
                })

    # Sắp xếp các peptide theo độ hydrophobic, net_charge và chiều dài
    sorted_results = sorted(results, key=lambda x: (x['hydrophobic_%'], x['net_charge'], x['length']), reverse=True)

    top = []
    for res in sorted_results:
        if not any(res['start'] <= existing['end'] and res['end'] >= existing['start'] for existing in top):
            top.append(res)
        elif len(top) < 5:
            for existing in top:
                if existing['start'] <= res['end'] and existing['end'] >= res['start']:
                    if res['length'] < existing['length']:
                        top.remove(existing)
                        top.append(res)
                        break
        if len(top) == 5:
            break
    return top

def process_fasta_file(input_path, output_path, resume_from_line=0, batch_size=1000):
    existing_sequences = set()  # Khởi tạo `existing_sequences` trước khi sử dụng
    current_id = 1  # Biến lưu trữ ID hiện tại

    # Đọc các chuỗi đã có trong file output (nếu có) và lưu vào set `existing_sequences`, đồng thời lấy ID cuối cùng
    if os.path.exists(output_path):
        with open(output_path, 'r') as f:
            lines = f.readlines()
            for i in range(1, len(lines), 2):  # Đọc các chuỗi peptide đã có (dòng sau dấu '>')
                existing_sequences.add(lines[i].strip())
            # Lấy ID cuối cùng đã ghi vào file
            last_id_line = lines[-2] if len(lines) > 1 else None
            if last_id_line:
                current_id = int(last_id_line.split()[0].strip('>S_')) + 1  # Cập nhật ID tiếp theo

    sequence = ''
    line_count = resume_from_line  # Dòng hiện tại đã xử lý
    batch_to_write = []  # Danh sách lưu chuỗi peptide cần ghi vào file theo batch

    with open(input_path, 'r') as f:
        for current_line_num, line in enumerate(f):
            if current_line_num < resume_from_line:
                continue

            line = line.strip()
            if line.startswith('>'):
                # Khi gặp tiêu đề (dòng bắt đầu bằng '>')
                if sequence:
                    # Kiểm tra chuỗi peptide có độ dài <= 50
                    if len(sequence) <= 50:
                        if sequence not in existing_sequences:
                            batch_to_write.append(f">S_{current_line_num + 1} nAMP\n{sequence}\n")
                            existing_sequences.add(sequence)  # Thêm vào existing_sequences để kiểm tra các lần sau
                    else:
                        # Nếu chuỗi dài hơn 50, tách thành các peptide con và kiểm tra
                        top_segments = analyze_peptide_top5(sequence)
                        for seg in top_segments:
                            seq = seg['sequence']
                            if seq not in existing_sequences:
                                batch_to_write.append(f">S_{current_line_num + 1} nAMP\n{seq}\n")
                                existing_sequences.add(seq)  # Thêm vào existing_sequences để kiểm tra các lần sau
                sequence = ''
            else:
                sequence += line

            # Cập nhật số dòng đã xử lý
            line_count = current_line_num + 1  # Cập nhật dòng hiện tại đã xử lý

            # Khi batch đã đủ 5000 peptide, ghi vào file
            if len(batch_to_write) >= batch_size:
                with open(output_path, 'a') as f_output:
                    f_output.writelines(batch_to_write)
                batch_to_write.clear()  # Làm trống batch sau khi ghi
                with open('line_count.txt', 'w') as f_count:
                    f_count.write(str(line_count))
                print(f"Đã xử lý dòng: {line_count}")  # In ra số dòng hiện tại

    # Ghi các peptide còn lại vào file sau khi xử lý hết (nếu có batch còn lại)
    if batch_to_write:
        with open(output_path, 'a') as f_output:
            f_output.writelines(batch_to_write)

# Ví dụ sử dụng hàm
current_line = 0  # Bắt đầu từ dòng đầu tiên

# Tiến hành xử lý tệp và tiếp tục từ dòng đã xử lý trước đó
process_fasta_file('namp_not_genbank.fasta', 'separated_namp_not_genbank.fasta', resume_from_line=current_line)